<a href="https://colab.research.google.com/github/lucas-bruzzone/Recipes-Dataset-64k-Dishes/blob/main/Dishes_Recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Download the dataset


In [ ]:
import os
import shutil

kaggle_dir = os.path.expanduser('~/.kaggle')
os.makedirs(kaggle_dir, exist_ok=True)

kaggle_json_src = '/content/drive/MyDrive/Colab Notebooks/Recipes Dataset : 64k Dishes/kaggle.json'
kaggle_json_dest = os.path.join(kaggle_dir, 'kaggle.json')
shutil.copy(kaggle_json_src, kaggle_json_dest)

os.chmod(kaggle_json_dest, 0o600)

In [ ]:
dataset_name = "prashantsingh001/recipes-dataset-64k-dishes"
kagglehub.dataset_download(dataset_name)

Using Colab cache for faster access to the 'recipes-dataset-64k-dishes' dataset.


'/kaggle/input/recipes-dataset-64k-dishes'

In [ ]:
import os
import shutil

source_dir = '/kaggle/input/recipes-dataset-64k-dishes'
destination_dir = '/content/drive/MyDrive/Colab Notebooks/Recipes Dataset : 64k Dishes/dataset'

os.makedirs(destination_dir, exist_ok=True)

# Copy the contents of the source directory to the destination directory
for item in os.listdir(source_dir):
    s = os.path.join(source_dir, item)
    d = os.path.join(destination_dir, item)
    # Use copy instead of move
    shutil.copy2(s, d)

print(f"Dataset copied from {source_dir} to {destination_dir}")

Dataset copied from /kaggle/input/recipes-dataset-64k-dishes to /content/drive/MyDrive/Colab Notebooks/Recipes Dataset : 64k Dishes/dataset


In [ ]:
import os

dataset_dir = '/content/drive/MyDrive/Colab Notebooks/Recipes Dataset : 64k Dishes/dataset'
print(os.listdir(dataset_dir))

['1_Recipe_csv.csv', '2_Recipe_json.json']


In [ ]:
import pandas as pd

dataset_path = '/content/drive/MyDrive/Colab Notebooks/Recipes Dataset : 64k Dishes/dataset/1_Recipe_csv.csv'
df = pd.read_csv(dataset_path)
display(df.head())

,recipe_title,category,subcategory,description,ingredients,directions,num_ingredients,num_steps
0,Air Fryer Potato Slices with Dipping Sauce,Air Fryer Recipes,Air Fryer Recipes,"These air fryer potato slices, served with a b...","[""3/4 cup ketchup"", ""1/2 cup beer"", ""1 tablesp...","[""Combine ketchup, beer, Worcestershire sauce,...",9,5
1,Gochujang Pork Belly Bites,Air Fryer Recipes,Air Fryer Recipes,These gochujang pork belly bites are sweet and...,"[""1 pound pork belly"", ""1/4 cup gochujang"", ""2...","[""Preheat an air fryer to 400 degrees F (200 d...",5,4
2,3-Ingredient Air Fryer Everything Bagel Chicke...,Air Fryer Recipes,Air Fryer Recipes,These 3-ingredient air fryer everything bagel ...,"[""1 \u00bc pounds chicken tenders"", ""1 tablesp...","[""Gather all ingredients. Preheat an air fryer...",3,4
3,Air Fryer Everything Bagel Chicken Cutlets,Air Fryer Recipes,Air Fryer Recipes,These air fryer everything bagel chicken cutle...,"[""4 chicken cutlets (about 1 pound total)"", ""s...","[""Preheat an air fryer to 400 degrees F (200 d...",9,9
4,Air Fryer Honey Sriracha Salmon Bites,Air Fryer Recipes,Air Fryer Recipes,These air fryer honey Sriracha salmon bites ar...,"[""1 tablespoon soy sauce"", ""1 tablespoon honey...","[""Preheat an air fryer to 400 degrees F (200 d...",5,5
